In [1]:
! pip install selenium

In [2]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver
from selenium import webdriver

# Importing required Exceptions which needs to be handled
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException

# Importing requests
import requests

from selenium.webdriver.common.by import By

QUES 1: Write a python program which searches all the product under a particular product from www.amazon.in. 
    The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’.
    Then search for guitars.


In [3]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.amazon.in')

In [ ]:
product_name=input("Which amazon product do you have in mind ?:")

In [ ]:
# give some time to load the webpage
time.sleep(2)

# get web element for amazon search bar
search_wbe=driver.find_element(By.ID,"nav-bb-search")
#enter the product into search bar
search_wbe.send_keys(product_name,Keys.ENTER)

# give some time to load the search results
time.sleep(2)

QUES 2:In the above question, now scrape the following details of each product listed in first 3 pages
    of your search results and save it in a data frame and csv. 
    In case if any product has less than 3 pages in search results then scrape all the products available
    under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.amazon.in')

In [ ]:
product_name=input("Which amazon product do you have in mind ?:")

In [ ]:
# give some time to load the webpage
time.sleep(2)

# get web element for amazon search bar
search_gui=driver.find_element(By.ID,"nav-bb-search")
#enter the product into search bar
search_gui.send_keys(product_name,Keys.ENTER)

# give some time to load the search results
time.sleep(2)

In [ ]:
def fetch_product():
# first get all product tiles from current webpage
#get product tiles webelements
global tiles_gui
tiles_gui=[]
tiles_gui.clear()
    
tiles_gui=driver.find_elements(By.XPATH,"//div[contains(@data-asin,'B0') and @data-component-type='s-search-result']")
    
# click on each tile one by one and get product info
for i in range(0,len(tiles_gui)):
    
        temp_gui=tiles_gui[i]
        #add time
        time.sleep(2)
        temp_gui.click()
        try:
            # switch to new window
            driver.switch_to.window(driver.window_handles[1])
        except IndexError:
            temp_gui=tiles_gui[i]
            time.sleep(2)
            temp_gui.click()
            driver.switch_to.window(driver.window_handles[1])
# add timer
time.sleep(3)
        
        
# get "enter pincode" button webelement
pincode_gui=driver.find_element(By.XPATH,"//div[@id='contextualIngressPtLabel_deliveryShortLine']")
        
if pincode_gui.text=='Select delivery location':
            #click on 'enter pincode' button
            pincode_gui.click()
            #add time to load pincode sub-window
            time.sleep(2)
            #get the webelement for pincode entry bar
            pin_bar_gui=driver.find_element(By.XPATH,"//input[@class='GLUX_Full_Width a-declarative']")
            #enter into pincode bar
            pin_bar_gui.send_keys('110091')
            #get webelement for pincode 'Apply' button
            apply_btn=driver.find_element(By.XPATH,"//input[@aria-labelledby='GLUXZipUpdate-announce']")
            #click on pincode 'Apply' button
            apply_btn.click()
            #add time to load the webpage after pincode enter
            time.sleep(3)
#===================================================== 
time.sleep(3)

# calling the function to fetch data from current webpage
fetch_data()
        
# give some time to fetch data
time.sleep(3)
        
# close the particular product tab
driver.close()
        
# switch driver to main window with search results
driver.switch_to.window(driver.window_handles[0])
        
time.sleep(2)

In [ ]:
def fetch_data():
"""this function gets required data from the current product webpage"""
    
# get brand name
brand_gui=driver.find_element(By.XPATH,"//span[@id='productTitle']")
brands.append(brand_gui.text.split(' ')[0])

# get name of the product
products.append(brand_gui.text)

# get the rating
    try:
        rating_gui=driver.find_element(By.XPATH,"//span[@data-hook='rating-out-of-text']") # only first instance of webelement from this page is required
        ratings.append(rating_gui.text)
    except NoSuchElementException:
        ratings.append('--')

# get number of ratings
    try:
        number_ratings_gui=driver.find_element(By.XPATH,"//a[@id='acrCustomerReviewLink']//following::span")
        number_ratings.append(number_ratings_gui.text.split(" ")[0])
    except NoSuchElementException:
        number_ratings.append('--')
 
 # get price 
    try:
        price_gui=driver.find_element(By.XPATH,"//span[@id='priceblock_ourprice']")
        prices.append(price_gui.text)
    except NoSuchElementException:
        try:
            price_gui=driver.find_element(By.XPATH,"//span[@id='priceblock_dealprice']")
            prices.append(price_gui.text+"(Deal price)")
        except NoSuchElementException:
            try:
                price_gui=driver.find_element(By.XPATH,"//span[@class='a-price-whole']")
                prices.append(price_gui.text+"(Prime member price)")
            except NoSuchElementException:
                prices.append('--')
    
# get return/exchange data
    try:
        xchange_gui=driver.find_element(By.XPATH,"//a[@class='a-size-small a-link-normal a-text-normal']")
        xchanges.append(xchange_gui.text)
    except NoSuchElementException:
        xchanges.append('--')
         

# get expected delivery webelement
    try:
        xp_del_gui=driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']")
        if xp_del_gui.text!='':
            xp_deliveries.append(xp_del_gui.text)
        else:
            xp_deliveries.append("Currently not available")
    except NoSuchElementException:
        xp_deliveries.append('--')
         
 # get product availability
    try:
        prod_avail_gui=driver.find_element(By.XPATH,"//div[@id='availability']")
        prod_avail.append(prod_avail_gui.text)
    except NoSuchElementException:
        prod_avail.append('--')     

# get other details
    try:
        other_details_gui=driver.find_element(By.XPATH,"//div[@id='featurebullets_feature_div']")
        other_details.append(other_details_gui.text.replace("\n","."))
    except NoSuchElementException:
        other_details.append('--')
   
 # get product url
    prod_url.append(driver.current_url)

In [ ]:
brands=[]
brands.clear()

products=[]
products.clear()

ratings=[]
ratings.clear()

number_ratings=[]
number_ratings.clear()

prices=[]
prices.clear()

xchanges=[]
xchanges.clear()

xp_deliveries=[]
xp_deliveries.clear()

prod_avail=[]
prod_avail.clear()

other_details=[]
other_details.clear()

prod_url=[]
prod_url.clear()

page_urls=[]
page_urls.clear()
input_prompt=input("Which amazon product do you have in mind ?:")

amzn_search(input_prompt)

#add time
time.sleep(3)

# get web element for webpage number button
page_guigui=driver.find_elements(By.XPATH,"//ul[@class='a-pagination']//following::a")

# get urls of 1st 3 pages
for i in page_gui:
    temp_url= i.get_attribute('href')
    page_urls.append(temp_url)
    if len(page_urls)==3:
        break

counter=1
# get product info by iterating over each page
for url in page_urls:
    if counter == 1: # first page check
        time.sleep(2)
        fetch_product() # get webelemets for product titles, then click on each title and open the product in new tab and get info.
        counter=2
    elif counter != 1: # first page scraping complete
        time.sleep(2)
        # open the next webpage in current tab
        driver.get(url)
        
        # give time to load
        time.sleep(3)
        
        # fetch data from current webpage
        fetch_product()
        
        # time
        time.sleep(3)

In [ ]:
len(brands)

In [ ]:
pd.set_option('display.max_colwidth',None)

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
amzn_3pg

QUES 3:Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

FRUITS PICTURES

In [21]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.google.co.in/imghp?hl=en&ogbl')
time.sleep(5)

In [27]:
search_images=input("Enter an element: ")

Enter an element: fruits


In [28]:
# get web element for search button
search_bar=driver.find_element(By.CLASS_NAME,"a4bIc")
#enter text into search bar
search_bar.send_keys(search_images)
# give time to load the images page
time.sleep(5)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF786DE1F52+60322]
	(No symbol) [0x00007FF786D5CEC9]
	(No symbol) [0x00007FF786C17CE9]
	(No symbol) [0x00007FF786C61093]
	(No symbol) [0x00007FF786C5F20B]
	(No symbol) [0x00007FF786C8C21A]
	(No symbol) [0x00007FF786C5ADB6]
	(No symbol) [0x00007FF786C8C430]
	(No symbol) [0x00007FF786CABC80]
	(No symbol) [0x00007FF786C8BFC3]
	(No symbol) [0x00007FF786C59617]
	(No symbol) [0x00007FF786C5A211]
	GetHandleVerifier [0x00007FF7870F94AD+3301629]
	GetHandleVerifier [0x00007FF7871436D3+3605283]
	GetHandleVerifier [0x00007FF787139450+3563680]
	GetHandleVerifier [0x00007FF786E94326+790390]
	(No symbol) [0x00007FF786D6750F]
	(No symbol) [0x00007FF786D63404]
	(No symbol) [0x00007FF786D63592]
	(No symbol) [0x00007FF786D52F9F]
	BaseThreadInitThunk [0x00007FFFDB14257D+29]
	RtlUserThreadStart [0x00007FFFDC3CAA48+40]


In [24]:

# get webelement for 1st image
first_image_wbe=driver.find_element(By.XPATH,"//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element(By.XPATH,"//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements(By.XPATH,"//div[@class='bRMDJf islir']//following::img")
# loop for 10 webelements
for i in range(10):
    images[i].screenshot('F:/fruit_pics/fruits_'+str(i)+'.png')

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF786DE1F52+60322]
	(No symbol) [0x00007FF786D5CEC9]
	(No symbol) [0x00007FF786C17CE9]
	(No symbol) [0x00007FF786C61093]
	(No symbol) [0x00007FF786C5F20B]
	(No symbol) [0x00007FF786C8C21A]
	(No symbol) [0x00007FF786C5ADB6]
	(No symbol) [0x00007FF786C8C430]
	(No symbol) [0x00007FF786CABC80]
	(No symbol) [0x00007FF786C8BFC3]
	(No symbol) [0x00007FF786C59617]
	(No symbol) [0x00007FF786C5A211]
	GetHandleVerifier [0x00007FF7870F94AD+3301629]
	GetHandleVerifier [0x00007FF7871436D3+3605283]
	GetHandleVerifier [0x00007FF787139450+3563680]
	GetHandleVerifier [0x00007FF786E94326+790390]
	(No symbol) [0x00007FF786D6750F]
	(No symbol) [0x00007FF786D63404]
	(No symbol) [0x00007FF786D63592]
	(No symbol) [0x00007FF786D52F9F]
	BaseThreadInitThunk [0x00007FFFDB14257D+29]
	RtlUserThreadStart [0x00007FFFDC3CAA48+40]


CAR PICTURES

In [30]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.google.co.in/imghp?hl=en&ogbl')
time.sleep(5)

In [42]:

# get web element for search button
search_wbe=driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea")
#enter text into search bar
search_wbe.send_keys('cars',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element(By.XPATH,"//div[class="H8Rx8c"]//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element(By.XPATH,"//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements(By.XPATH,"//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/car_pics/car_'+str(i)+'.png')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3124965196.py, line 8)

MACHINE LEARNING PICTURES

In [38]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('Machine Learning',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/ml_pics/ml_'+str(i)+'.png')

NameError: name 'open_url' is not defined

QUES 4 :Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
from selenium import webdriver
import pandas as pd

driver=webdriver.Chrome()

# fetching url
driver.get(' www.flipkart.com ')

# Search for the smartphone
search_box = driver.find_element(By.XPATH,'//input[@title="Search for products, brands and more"]')
search_box.send_keys('Oneplus Nord')  # Replace 'Oneplus Nord' with the desired smartphone name
search_box.submit()

# Scrape the details from the search results
details = []
products = driver.find_elements(By.XPATH,'//div[@class="_1AtVbE"]')
for product in products:
    product_url = product.find_element(By.XPATH,'.//a').get_attribute('href')
    name = product.find_element(By.XPATH,'.//a').text
    details.append({
    'Product URL': product_url,
    'Smartphone Name': name,
    # Add other details here using similar methods
  })

# Create a dataframe and save the results to CSV
df = pd.DataFrame(details)
df.to_csv('smartphones.csv', index=False)

# Close the webdriver
driver.quit()

QUES 5:Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [7]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.google.com/maps/')

In [ ]:
search_city=input("Enter a location: ")

In [11]:
# get search bar webelement
search_bar=driver.find_element(By.XPATH,'//*[@id="searchboxinput"]')
# send city name
search_bar.send_keys(search_city,Keys.ENTER)
time.sleep(10)
current_url=driver.current_url
longitude=current_url.split('@')[1].split(',')[0]
lattitude=current_url.split('@')[1].split(',')[1]
print(prompt," Longitude: ",longitude,'"N')
print(prompt," Lattitude: ",lattitude,'"E')

Enter a location: delhi
mumbai  Longitude:  21.0521522 "N
mumbai  Lattitude:  73.0521039 "E


QUES 6: Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.digit.in')

# click on 'Best gaming laptops' link
driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a').click()

time.sleep(3)

# get laptop names
names=[]
names.clear()

names_wbe=driver.find_elements(By.XPATH,"//h3")

for i in names_wbe:
    names.append(i.text)
    
time.sleep(2)

# get specifications box weblements
spec_wbe=driver.find_elements(By.XPATH,"//div[@class='Spcs-details']")

specs=[]
specs.clear()

for wbe in spec_wbe:
    specs.append(wbe.text)

In [ ]:
names

In [ ]:
specs

In [ ]:
specs[0]

In [ ]:
temp_specs=[]
for specs in specs:
    temp_specs.append(specs.split('\n'))
temp_specs[0]

In [ ]:
temp_specs

In [ ]:
temp_specs[0][3][0:3]

In [ ]:
os=[]
dis=[]
processor=[]
memory=[]
weight=[]
dimension=[]
graphic=[]
price=[]

In [ ]:
for j in temp_specs:
    if len(j)==8:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                price.append('--')
                continue
    elif len(j)==9:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pri':
                price.append(i.split(':')[-1])
                time.sleep(2)
                continue


In [ ]:
df_digit=pd.DataFrame({})
df_digit['Model Name']=names
df_digit['Operating System']=os
df_digit['Display']=dis
df_digit['Processor']=processor
df_digit['Memory']=memory
df_digit['Weight']=weight
df_digit['Dimension']=dimension
df_digit['Graphics Processor']=graphic
df_digit['Price']=price

In [ ]:
df_digit

In [ ]:
df_digit.to_csv('Best gaming laptops.csv',float_format='%.2f',index=False)

In [ ]:
df_temp=pd.read_csv('Best gaming laptops.csv')

In [ ]:
df_temp

QUES 7: Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
driver=webdriver.Chrome()

# fetching url
driver.get('https://www.forbes.com/billionaires/')
driver.maximize_window()
time.sleep(2)




# Create Empty list
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []



#Fetching Rank
rank_tag = driver.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")





#Fetching Name
name_tag = driver.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")





#Fetching Net Worth
netwrth_tag = driver.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")





#Fetching Age
age_tag = driver.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")





#Fetching Citizenship
cit_tag = driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")





#Fetching Source
src_tag = driver.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")





#Fetching Industry
ind_tag = driver.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")





# Length of all the coloumns
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)





#create Dataframe
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires



QUES 8:Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [ ]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)





driver.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver.maximize_window()
time.sleep(2)





i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1





comment = []
upvote = []
comment_time = []





comment_tag=(driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")





upvote_tag=(driver.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")





comment_time_tag=(driver.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")





video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video



QUES 9:Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32 (1)/chromedriver.exe")

In [ ]:
# getting the web page of mentioned url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(3)

In [ ]:
# locating the location search bar
search_bar = driver.find_element(By.ID,"search-input-field")

# entering London in search bar
search_bar.send_keys("London")

In [ ]:
# select London
London = driver.find_element(By.XPATH,"//ul[@id='predicted-search-results']//li[2]")
#clicking on button
London.click()

# do click on Let's Go button
search_btn = driver.find_element(By.ID,'search-button')
search_btn.click()

In [ ]:
# creating empty list & find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description = []
url = []

In [ ]:
# scraping the required informations
for i in driver.find_elements(By.XPATH,"//div[@class='pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(3)

# scraping  hostel name
    try:
        name = driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
        
        
# scraping distance from city centre
    try:
        dist = driver.find_elements(By.XPATH,"//div[@class='subtitle body-3']//a//span[1]")
        for i in name:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):   

# scraping privates from price
        try:
            pvt_price = driver.find_element(By.XPATH,"//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
   

        for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):          

        
# scraping dorms from price
        try:
            dorms = driver.find_element(By.XPATH,"//a[@class='prices']//div[2]/div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            
            
# scraping facilities
    try:
        fac1 = driver.find_elements(By.XPATH,"//div[@class='has-wifi']")
        fac2 = driver.find_elements(By.XPATH,"//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text)
    except NoSuchElementException:
        facilities.append('-')
     
            
#fetching url of each hostel
    p_url = driver.find_elements(By.XPATH,"//div[@class='prices-col']//a[2]")
    for i in p_url:
        url.append(i.get_attribute("href"))
        
for i in url:
    driver.get(i)
    time.sleep(3)
    

# scraping ratings
    try:
        rat = driver.find_element(By.XPATH,"//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
        
# scraping total review
    try:
        rws = driver.find_element(By.XPATH,"//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        
        
# fetching over all review
    try:
        overall = driver.find_element(By.XPATH,"//div[@class='keyword']//span")
        over_all.append(overall.text)
    except NoSuchElementException:
        over_all.append('-')
        
        
# fetching property description
    try:
        disc = driver.find_element(By.XPATH,"//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')
    
# do click on show more button for description
    try:
        driver.find_element(By.XPATH,"//a[@class='toggle-content']").click()
        time.sleep(4)
    except NoSuchElementException:
        pass   

In [ ]:
print(len(hostel_name),
len(distance),
len(pvt_prices),
len(dorms_price),
len(rating),
len(reviews),
len(over_all),
len(facilities),
len(description),
len(url))

In [ ]:
# creating DataFrame
Hostel = pd.DataFrame({})
Hostel['Hostel Name'] = hostel_name
Hostel['Distance from City Centre'] = distance
Hostel['Ratings'] = rating
Hostel['Total Reviews'] = reviews
Hostel['Overall Reviews'] = over_all
Hostel['Privates from Price'] = pvt_prices
Hostel['Dorms from Price'] = dorms_price
Hostel['Facilities'] = facilities[:74]
Hostel['Description'] = description
Hostel

In [ ]:
# saving the dataset to csv
Hostel.to_csv("London_Hostels.csv")

In [ ]:

driver.close()